In [1]:
import requests
from bs4 import BeautifulSoup
import json
import urllib
headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'}

In [2]:
#網址轉碼
def percentEncode(str):
    res = urllib.parse.quote(str, '')
    res = res.replace('+', '%20')
    res = res.replace('*', '%2A')
    res = res.replace('%7E', '~')
    return res

In [ ]:
#step1:以原本設定的params作為參數查詢，並列印出我們要的留言者、留言時間、留言內容(共10則)
#step2:從網站回傳的json檔中找出after_cursor這個參數，此參數作為連接下10則留言的關鍵參數
#step3:將after_cursor參數新增至params中，以新設定的params作為參數查詢，並列印出我們要的留言者、留言時間、留言內容(共10則)
#step4:重複step2與step3直到after_cursor==1時，停止

In [3]:
data={#'app_id':'241741912846250',
      'app_id':'140490219413038',
'limit': '10',
'__a': '1'
}
#範例:https://www.facebook.com/plugins/feedback.php?app_id=新聞的app_id&channel=不同新聞網站有不同的channel&color_scheme=light&href='+轉碼後的網址
#url='https://news.ltn.com.tw/news/politics/breakingnews/3039295'
url='https://news.ltn.com.tw/news/world/breakingnews/3086335'
x=percentEncode(url)
url='https://www.facebook.com/plugins/feedback.php?app_id=140490219413038&channel=https%3A%2F%2Fstaticxx.facebook.com%2Fconnect%2Fxd_arbiter.php%3Fversion%3D44%23cb%3Df3aa6df476891a%26domain%3Dnews.ltn.com.tw%26origin%3Dhttps%3A%2F%2Fnews.ltn.com.tw%2Ff1ecff2b16d5cec%26relation%3Dparent.parent&href='+x
resp=requests.get(url,headers=headers)
fbid=resp.text.split("commentIDs")[1].split(':["')[1].split('_')[0]
fbid

'3488869321187106'

In [5]:
#step1
params_0={
    'app_id': '140490219413038',
    'limit': '10',
    '__user': '0',
    '__a': '1',
    '__req': '2',
    '__beoa': '0',
    'dpr': '1',
    '__comet_req': '1',
    'locale': 'zh_TW',
}


count=1
while True:
    url='https://www.facebook.com/plugins/comments/async/'+fbid+'/pager/social/'
    session=requests.session()
    resp=session.post(url,data=params_0)
    text=json.loads(resp.text.split(';',3)[-1])
    comment_id=text['payload']['commentIDs']
    afterCursor=text['payload']['afterCursor']
    print('comment_id_count:'+str(len(comment_id)))
    print('afterCursor:'+afterCursor)
    
    for i in comment_id:
        text=json.loads(resp.text.split(';',3)[-1])
        authorid=text['payload']['idMap'][i]['authorID']
        commenter=text['payload']['idMap'][authorid]['name']
        context=text['payload']['idMap'][i]['body']['text']
        comment_time=text['payload']['idMap'][i]['timestamp']['text'].split(' ')[0].replace('年','-').replace("月",'-').replace("日",'')

        print(count,end=':')
        count+=1
        print(commenter)
        print(comment_time)
        print(context)
        print("-----------")
    
    #留言回覆
    #留言回覆若大於五則，需要post新網頁
    comment_reply=[]
    for i in comment_id:
        try:
            if text['payload']['idMap'][i]['public_replies']['totalCount'] !=0:
                comment_reply.append(text['payload']['idMap'][i]['public_replies']['commentIDs'])
        except:
            pass
    for j in comment_reply:
        for i in j:
            author_id=text['payload']['idMap'][i]['authorID']
            commenter=text['payload']['idMap'][author_id]['name']
            comment_time=text['payload']['idMap'][i]['timestamp']['text'].split(' ')[0]
            comment_time.replace('年','-').replace("月",'-').replace("日",'')
            print(count,end=':')
            count+=1
            print(commenter)
            print(comment_time)
            print(text['payload']['idMap'][i]['body']['text'].replace(u'\n',u''))#comment
            print("----------")
#step2 and step3 and step4

    params_0['after_cursor']=afterCursor
    
    if afterCursor=='1':
        break


comment_id_count:10
afterCursor:81
1:許建泓
2020-3-2
在說甚麼蠢話
那是因為習包子隱匿阿
其他國家的透明度是習包子國的千萬倍阿
-----------
2:你再嘴阿
2020-3-2
真的，都是外國傳染給中國，難怪中國要撤僑.............................(哄堂大笑)
-----------
3:Apollo Lee
2020-3-2
不要在奢望這組織了。這組織大多數人都已經被名聲，利益，遮蔽眼睛了。會有報應的。相信這些人的國家疫情會越來越嚴重的！
-----------
4:We Chen
2020-3-2
他國病例數是中國的九倍.....這種話怎麼說得出口，為了錢，連人類基本的道德及尊嚴都不要了嗎
-----------
5:Nengfu  Yang
2020-3-2
老共豢養的一隻阿狗，只會看主人臉色汪汪叫！
-----------
6:Xuan-ming Xu
2020-3-2
有差嗎，這組織的公信力早就喪失了
-----------
7:羅一凡
2020-3-2
這個人渣眼睛就只有人民幣，也難怪一再講x話！
-----------
8:Lin Sgo
2020-3-2
恭喜彈得賽，成功把瘟疫傳播全世界，讓中國不再孤單，這打賞應不少！
-----------
9:陳金和
2020-3-2
頭上三尺有神明 很快 奈及利亞會為他犧牲 就
-----------
10:Michael Chung
2020-3-2
甚麼賽的，狗奴才是說中國很棒還是他國很差？
-----------
comment_id_count:10
afterCursor:71
11:Yu-Fan Chang
2020-3-2
所以WHO這次根本就是個笑話...
-----------
12:尤保棠
2020-3-2
希望 痰得賽 身體健康不要得到武漢病毒！不要被習大大給傳染了~
-----------
13:Rosemary Su
2020-3-2
邪惡中共眷養的黑奴,是中共的大外宣
-----------
14:劉珍劭
2020-3-2
全世界都爆發增加，只有中國減少？
這種話由WHO高官講出來，可見這位官員的智商堪慮！
-----------
15:薛凌雲
2020-3-2
三句不離強國好......有沒有考慮入籍強

In [14]:
params_1={
    'app_id': '140490219413038',
    'limit': '10',
    '__user': '0',
    '__a': '1',
    '__req': '2',
    '__beoa': '0',
    'dpr': '1',
    '__comet_req': '0',
    'locale': 'zh_TW',
    '__sp': '1',
}
url='https://www.facebook.com/plugins/comments/async/comment/3562746710463425_3562946597110103/pager/'
session=requests.session()
resp=session.post(url,data=params_1)

In [21]:
text=json.loads(resp.text.split(';',3)[-1])
comment_id=text['payload']['commentIDs']
for i in comment_id:
    author_id=text['payload']['idMap'][i]['authorID']
    commenter=text['payload']['idMap'][author_id]['name']
    comment_time=text['payload']['idMap'][i]['timestamp']['text'].split(' ')[0]
    comment_time.replace('年','-').replace("月",'-').replace("日",'')
    print(commenter)
    print(comment_time)
    print(text['payload']['idMap'][i]['body']['text'].replace(u'\n',u''))#comment
    print("----------")

In [19]:
afterCursor=text['payload']['afterCursor']
params_1['after_cursor']=afterCursor

In [20]:
url='https://www.facebook.com/plugins/comments/async/comment/3562746710463425_3562946597110103/pager/'
session=requests.session()
resp=session.post(url,data=params_1)

In [22]:
len(comment_id)

0

In [ ]:
#留言回覆
#step1:若留言的回覆數量大於0，將comment_id放入陣列中
#step2:將comment_id放入會導向回覆內容的網址
#step3:將after_cursor參數抓出來，作為讀取下10則的查詢參數
#step4:若新讀取的comment_id為0，則代表沒有留言可以讀取(同時停止迴圈)